<a href="https://colab.research.google.com/github/esthy13/cil-intrusion-detection/blob/main/notebooks/cyber_project_margarita.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
import math
from torch.utils.data import Dataset
import numpy as np
import pandas as pd
import torch
import glob


# Download Original Dataset

In [13]:
!wget http://205.174.165.80/CICDataset/CIC-IDS-2017/Dataset/CIC-IDS-2017/CSVs/MachineLearningCSV.zip
!unzip MachineLearningCSV.zip

--2026-02-04 09:12:52--  http://205.174.165.80/CICDataset/CIC-IDS-2017/Dataset/CIC-IDS-2017/CSVs/MachineLearningCSV.zip
Connecting to 205.174.165.80:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 235102953 (224M) [application/zip]
Saving to: ‘MachineLearningCSV.zip.1’

MachineLearningCSV. 100%[===================>] 224.21M  14.9MB/s    in 28s     

2026-02-04 09:13:21 (7.92 MB/s) - ‘MachineLearningCSV.zip.1’ saved [235102953/235102953]

Archive:  MachineLearningCSV.zip
replace MachineLearningCVE/Wednesday-workingHours.pcap_ISCX.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: MachineLearningCVE/Wednesday-workingHours.pcap_ISCX.csv  
  inflating: MachineLearningCVE/Tuesday-WorkingHours.pcap_ISCX.csv  
  inflating: MachineLearningCVE/Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv  
  inflating: MachineLearningCVE/Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv  
  inflating: MachineLearningCVE/Monday-WorkingHours.pcap_ISCX.csv  
  inf

In [15]:
import pandas as pd
import numpy as np
import glob

In [16]:
csv_path = "MachineLearningCVE"
csv_files = glob.glob(f"{csv_path}/*.csv")

print(len(csv_files))
csv_files


8


['MachineLearningCVE/Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv',
 'MachineLearningCVE/Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv',
 'MachineLearningCVE/Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv',
 'MachineLearningCVE/Wednesday-workingHours.pcap_ISCX.csv',
 'MachineLearningCVE/Monday-WorkingHours.pcap_ISCX.csv',
 'MachineLearningCVE/Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv',
 'MachineLearningCVE/Friday-WorkingHours-Morning.pcap_ISCX.csv',
 'MachineLearningCVE/Tuesday-WorkingHours.pcap_ISCX.csv']

In [17]:
dfs = []
for f in csv_files:
    print("Loading:", f)
    dfs.append(pd.read_csv(f, encoding="latin1"))

df = pd.concat(dfs, ignore_index=True)
print(df.shape)


Loading: MachineLearningCVE/Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv
Loading: MachineLearningCVE/Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv
Loading: MachineLearningCVE/Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv
Loading: MachineLearningCVE/Wednesday-workingHours.pcap_ISCX.csv
Loading: MachineLearningCVE/Monday-WorkingHours.pcap_ISCX.csv
Loading: MachineLearningCVE/Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv
Loading: MachineLearningCVE/Friday-WorkingHours-Morning.pcap_ISCX.csv
Loading: MachineLearningCVE/Tuesday-WorkingHours.pcap_ISCX.csv
(2830743, 79)


In [18]:
df.columns = df.columns.str.strip()  # quita espacios raros
df = df.replace([np.inf, -np.inf], np.nan)
df = df.dropna()


In [ ]:
df.isna().sum().sum()
print(df.shape)
df['Label'].unique()


(2827876, 79)


array(['BENIGN', 'DDoS', 'Web Attack ï¿½ Brute Force',
       'Web Attack ï¿½ XSS', 'Web Attack ï¿½ Sql Injection', 'PortScan',
       'DoS slowloris', 'DoS Slowhttptest', 'DoS Hulk', 'DoS GoldenEye',
       'Heartbleed', 'Infiltration', 'Bot', 'FTP-Patator', 'SSH-Patator'],
      dtype=object)

In [19]:
import re

def map_attack(label: str) -> str:
    label = label.strip()

    patterns = [
        (r"^BENIGN$", "benign"),
        (r"^PortScan$", "portscan"),
        (r"^Bot$", "bot"),
        (r"^Infiltration$", "infiltration"),
        (r"^FTP-Patator$", "ftp-patator"),
        (r"^SSH-Patator$", "ssh-patator"),
        (r"^Heartbleed$", "heartbleed"),
        (r"^DDoS$", "ddos"),
        (r"Web\s*Attack", "web-attack"),
        (r"DoS", "dos"),
    ]

    for pattern, mapped_label in patterns:
        if re.search(pattern, label, re.IGNORECASE):
            return mapped_label

    return "unknown"


In [20]:
df["Label"] = df["Label"].apply(map_attack)
print(df['Label'].unique())

['benign' 'ddos' 'web-attack' 'portscan' 'dos' 'heartbleed' 'infiltration'
 'bot' 'ftp-patator' 'ssh-patator']


# Export Datasets

In [ ]:
import os

OUTPUT_DIR = "CIL_data"
os.makedirs(OUTPUT_DIR, exist_ok=True)


In [ ]:
CLASSES = sorted(df["Label"].unique())
print(CLASSES)


['benign', 'bot', 'ddos', 'dos', 'ftp-patator', 'heartbleed', 'infiltration', 'portscan', 'ssh-patator', 'web-attack']


In [ ]:
'''for cls in CLASSES:
    cls_df = df[df["Label"] == cls]

    out_path = f"{OUTPUT_DIR}/{cls}.csv"
    cls_df.to_csv(out_path, index=False)

    print(f"Saved {cls}: {cls_df.shape}")'''


import os
from sklearn.model_selection import train_test_split

# Config
OUTPUT_DIR = "output"
TRAIN_DIR = os.path.join(OUTPUT_DIR, "train")
TEST_DIR = os.path.join(OUTPUT_DIR, "test")
TEST_RATIO = 0.2
SEED = 42

os.makedirs(TRAIN_DIR, exist_ok=True)
os.makedirs(TEST_DIR, exist_ok=True)

CLASSES = df["Label"].unique()

for cls in CLASSES:
    cls_df = df[df["Label"] == cls]

    train_df, test_df = train_test_split(
        cls_df,
        test_size=TEST_RATIO,
        random_state=SEED,
        shuffle=True,
        stratify=None  # ya estamos filtrando por clase
    )

    train_path = os.path.join(TRAIN_DIR, f"{cls}.csv")
    test_path = os.path.join(TEST_DIR, f"{cls}.csv")

    train_df.to_csv(train_path, index=False)
    test_df.to_csv(test_path, index=False)

    print(
        f"Class {cls:15s} | "
        f"Train: {train_df.shape[0]:6d} | "
        f"Test: {test_df.shape[0]:6d}"
    )



Class benign          | Train: 1817056 | Test: 454264
Class ddos            | Train: 102420 | Test:  25605
Class web-attack      | Train:   1744 | Test:    436
Class portscan        | Train: 127043 | Test:  31761
Class dos             | Train: 201369 | Test:  50343
Class heartbleed      | Train:      8 | Test:      3
Class infiltration    | Train:     28 | Test:      8
Class bot             | Train:   1564 | Test:    392
Class ftp-patator     | Train:   6348 | Test:   1587
Class ssh-patator     | Train:   4717 | Test:   1180


In [ ]:
# Dataset Class

In [21]:
import glob
import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset


class IDSBaseDataset(Dataset):
    def __init__(self, csv_dir, class_to_idx):

        """
        Base IDS dataset.
        Loads all CSVs once and keeps data indexable.
        """
        super().__init__()
        csv_paths = glob.glob(f"{csv_dir}/*.csv")
        assert len(csv_paths) > 0, "No CSV files found"

        df = pd.concat([pd.read_csv(p) for p in csv_paths], ignore_index=True)

        self.class_to_idx = class_to_idx
        self.idx_to_class = {v: k for k, v in class_to_idx.items()}

        self.features = df.drop(columns=["Label"]).values.astype(np.float32)
        self.labels = np.array(
            [self.class_to_idx[label] for label in df["Label"].values],
            dtype=np.int64
        )

        # by default: use all samples
        self.indices = np.arange(len(self.labels))

    def __len__(self):
        return len(self.indices)

    def __getitem__(self, idx):
        real_idx = self.indices[idx]
        x = self.features[real_idx]
        y = self.labels[real_idx]
        return torch.tensor(x), torch.tensor(y)

In [ ]:
def sample_memory_stratified(dataset, memory_size):
    """
    Stratified memory sampling.
    Each class contributes (approximately) the same number of samples.
    """

    if memory_size <= 0:
        return np.empty(0, dtype=int)

    labels = dataset.labels
    indices = dataset.indices

    classes = np.unique(labels[indices])
    n_classes = len(classes)

    if n_classes == 0:
        return np.empty(0, dtype=int)

    base_quota = memory_size // n_classes
    remainder = memory_size % n_classes

    memory_indices = []

    for c in classes:
        class_idx = indices[labels[indices] == c]

        if len(class_idx) == 0:
            continue

        k = min(base_quota, len(class_idx))
        chosen = np.random.choice(class_idx, k, replace=False)
        memory_indices.extend(chosen)

    # distribute remaining samples (one extra per class if possible)
    if remainder > 0:
        for c in classes:
            if remainder == 0:
                break

            class_idx = indices[labels[indices] == c]
            available = np.setdiff1d(class_idx, memory_indices)

            if len(available) == 0:
                continue

            extra = np.random.choice(available, 1)
            memory_indices.append(extra[0])
            remainder -= 1

    return np.asarray(memory_indices, dtype=int)


In [ ]:
import numpy as np
from torch.utils.data import Subset

def build_task(dataset, task_class_names, buffer_size, old_task_dataset=None):
    """
    dataset: IDSContinualDataset (train o test)
    task_class_names: lista de strings, ej. ['benign', 'ddos']
    """-

    # convertir nombres a índices
    task_class_ids = [
        dataset.class_to_idx[c] for c in task_class_names
    ]

    # old task labels
    if old_task_dataset is None:
        old_task_labels = np.array([], dtype=np.int64)
    else:
        old_task_labels = np.unique(old_task_dataset.labels)

    # seleccionar índices
    task_indices = np.where(
        np.isin(dataset.labels, task_class_ids)
    )[0]

    # Buffer

    return Subset(dataset, task_indices)


# Model

In [ ]:
import torch.nn as nn


class CILModel(nn.Module):
    def __init__(self, input_size, num_classes):
        super().__init__()

        # simple MLP feature extractor
        self.feature_extractor = nn.Sequential(
            nn.Linear(input_size, 128),
            nn.ReLU(),
            nn.Linear(128, 256),
            nn.ReLU()
        )

        self.classifier = nn.Linear(256, num_classes)

    def forward(self, x):
        features = self.feature_extractor(x)
        out = self.classifier(features)
        return out

    def increment_classifier(self, num_new_classes):
        """Increment the classifier to accommodate new classes."""
        device = next(self.parameters()).device

        old_classifier = self.classifier
        old_num_classes = old_classifier.out_features
        new_num_classes = old_num_classes + num_new_classes

        # create a new linear layer with the updated number of classes
        new_classifier = nn.Linear(old_classifier.in_features, new_num_classes)

        # copy the weights and biases from the old classifier
        new_classifier.weight.data[:old_num_classes] = old_classifier.weight.data
        new_classifier.bias.data[:old_num_classes] = old_classifier.bias.data

        # replace classifier
        self.classifier = new_classifier.to(device)

# ICARL

In [ ]:
import copy
from abc import ABC, abstractmethod
import torch
import torch.optim as optim


def create_optimizer(model, lr):
    return optim.Adam(model.parameters(), lr=lr)


class BaseStrategy(ABC):
    def __init__(self, model, criterion, batch_size, lr, new_task_lr):
        self.model = model
        self.criterion = criterion
        self.batch_size = batch_size
        self.lr = lr
        self.new_task_lr = new_task_lr
        self.optimizer = create_optimizer(self.model, self.lr)
        self.seen_classes = set()

    def train(self, train_loader, task_id, val_loader, epochs, patience):
        """
        Main training loop with validation and early stopping.
        """
        best_val_loss = float("inf")
        patience_counter = 0
        best_model_state = copy.deepcopy(self.model.state_dict())

        for epoch in range(epochs):
            self._train_one_epoch(train_loader, task_id, epoch, epochs)

            val_loss = self._validate_one_epoch(val_loader)
            print(f"Epoch [{epoch + 1}/{epochs}], Validation Loss: {val_loss:.4f}")

            if val_loss < best_val_loss:
                best_val_loss = val_loss
                patience_counter = 0
                best_model_state = copy.deepcopy(self.model.state_dict())
            else:
                patience_counter += 1
                if patience_counter >= patience:
                    print(f"Early stopping at epoch {epoch + 1}")
                    break

        self.model.load_state_dict(best_model_state)

    @abstractmethod
    def classify(self, X_eval_tensor):
        """
        Classifies the given inputs using the current model.
        """
        raise NotImplementedError

    @abstractmethod
    def _get_replay_loader(self, task_id):
        """
        Returns the replay loader for the strategy.
        """
        raise NotImplementedError

    @abstractmethod
    def _train_one_epoch(self, train_loader, task_id, epoch, epochs):
        """
        Trains the model for one epoch on the provided training data.
        """
        raise NotImplementedError

    def _validate_one_epoch(self, val_loader):
        """
        Performs one epoch of validation.
        """
        self.model.eval()
        total_val_loss = 0.0
        total_samples = 0
        with torch.no_grad():
            for features, labels in val_loader:
                outputs = self.model(features)
                loss = self.criterion(outputs, labels)
                batch_size = features.shape[0]
                total_val_loss += loss.item() * batch_size
                total_samples += batch_size

        return total_val_loss / total_samples

    def after_task_training(
        self, train_dataset, X_train_tensor, y_train_tensor, new_classes
    ):
        """
        Hook called after training on a task is completed.
        """
        self.model.eval()
        self.seen_classes.update(new_classes)

    def after_task_model_update(self, **kwargs):
        """
        Hook called after the model is updated for a new task.
        """
        self.optimizer = create_optimizer(self.model, self.new_task_lr)

In [ ]:
from itertools import cycle
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader, TensorDataset


class iCaRL(BaseStrategy):
    def __init__(
        self,
        model,
        criterion,
        batch_size,
        lr,
        new_task_lr,
        buffer_size,
        device,
        distillation_lambda,
    ):
        super().__init__(model, criterion, batch_size, lr, new_task_lr)
        self.buffer_size = buffer_size
        self.device = device
        self.distillation_lambda = distillation_lambda

        self.replay_buffer = []
        self.exemplars = {}
        self.exemplar_means = {}

    def _get_replay_loader(self, task_id):
        if task_id == 0 or not self.replay_buffer:
            return None

        # stack the data and labels from the memory buffer into tensors
        replay_features = torch.stack([item[0] for item in self.replay_buffer])
        replay_labels = torch.stack([item[1] for item in self.replay_buffer])
        replay_logits = torch.stack([item[2] for item in self.replay_buffer])

        replay_dataset = TensorDataset(replay_features, replay_labels, replay_logits)

        return DataLoader(
            dataset=replay_dataset, batch_size=self.batch_size, shuffle=True
        )

    def _train_one_epoch(self, train_loader, task_id, epoch, epochs):
        # parallel iteration over train and replay loaders
        train_iter = iter(train_loader)

        replay_loader = self._get_replay_loader(task_id)
        if replay_loader is not None:
            replay_iter = cycle(replay_loader)
        else:
            replay_iter = None

        # loop until the training data is exhausted
        for i in range(len(train_loader)):
            self.optimizer.zero_grad()
            loss = 0.0

            # training on new data
            try:
                features, labels = next(train_iter)
                outputs = self.model(features)
                loss += self.criterion(outputs, labels)
            except StopIteration:
                pass

            if replay_iter is not None:
                # training on replay data with distillation
                try:
                    replay_features, replay_labels, teacher_logits = next(replay_iter)

                    student_logits = self.model(replay_features)
                    loss += self.criterion(student_logits, replay_labels)

                    # distillation loss should only be calculated on the logits of old classes
                    loss += self.distillation_lambda * self._distillation_loss(
                        student_logits, teacher_logits
                    )

                except StopIteration:
                    pass

            if loss != 0.0:
                loss.backward()
                self.optimizer.step()

            if (i + 1) % 200 == 0:
                print(
                    f"Task [{task_id + 1}], Epoch [{epoch + 1}/{epochs}], "
                    f"Step [{i + 1}/{len(train_loader)}], Loss: {loss.item():.4f}"
                )

    def classify(self, X_eval_tensor):
        preds = []

        with torch.no_grad():
            eval_features = self.model.feature_extractor(X_eval_tensor)
            for feature in eval_features:
                closest_class = None
                closest_distance = float("inf")

                for c in self.exemplar_means:
                    # find the closest class mean
                    dist = torch.norm(feature - self.exemplar_means[c])
                    if dist < closest_distance:
                        closest_distance = dist
                        closest_class = c
                preds.append(closest_class)

        return torch.tensor(preds).to(self.device)

    def after_task_training(
        self, train_dataset, X_train_tensor, y_train_tensor, new_classes
    ):
        super().after_task_training(
            train_dataset, X_train_tensor, y_train_tensor, new_classes
        )
        self._update_memory(X_train_tensor, y_train_tensor, new_classes)
        self._compute_class_means()

    def _update_memory(self, X_train_tensor, y_train_tensor, new_classes):
        num_total_classes = len(self.seen_classes)
        exemplars_per_class = self.buffer_size // num_total_classes

        # trim exemplars for old classes if necessary
        for c in self.exemplars:
            if c not in new_classes:
                self.exemplars[c] = self.exemplars[c][:exemplars_per_class]

        self.model.eval()

        for c in new_classes:
            indices = (y_train_tensor == c).nonzero(as_tuple=True)[0]
            current_features = X_train_tensor[indices]
            current_labels = y_train_tensor[indices]

            # if we have old exemplars for this class, combine them with the new data
            if c in self.exemplars and len(self.exemplars[c]) > 0:
                old_features = torch.stack([item[0] for item in self.exemplars[c]])
                old_labels = torch.stack([item[1] for item in self.exemplars[c]])

                candidate_features = torch.cat((old_features, current_features), dim=0)
                candidate_labels = torch.cat((old_labels, current_labels), dim=0)
            else:
                candidate_features = current_features
                candidate_labels = current_labels

            if len(candidate_features) == 0:
                continue

            with torch.no_grad():
                feature = self.model.feature_extractor(candidate_features)

            # normalize features
            feature = torch.nn.functional.normalize(feature, p=2, dim=1)

            # compute class mean
            class_mean = torch.mean(feature, dim=0)

            # herding
            selected_indices = []
            current_sum = torch.zeros_like(class_mean)

            current_k = min(exemplars_per_class, len(feature))

            for k in range(current_k):
                # we want to select the exemplar that, when added to the current sum,
                # brings the average closest to the class mean
                target = class_mean * (k + 1) - current_sum

                distances = torch.norm(feature - target, dim=1)

                if len(selected_indices) > 0:
                    # avoid selecting the same exemplar again
                    distances[selected_indices] = float("inf")

                # select the best exemplar
                best_idx = int(torch.argmin(distances).item())

                selected_indices.append(best_idx)
                current_sum += feature[best_idx]

            self.exemplars[c] = [
                (candidate_features[idx], candidate_labels[idx])
                for idx in selected_indices
            ]

        # update replay buffer
        self.replay_buffer = []
        with torch.no_grad():
            for c in self.exemplars:
                for feature, label in self.exemplars[c]:
                    logits = self.model(feature.unsqueeze(0)).squeeze(0)
                    self.replay_buffer.append((feature, label, logits))

    def _distillation_loss(self, student_logits, teacher_logits, T=2.0):
        num_teacher_cls = teacher_logits.shape[1]

        student_dist = F.log_softmax(student_logits[:, :num_teacher_cls] / T, dim=1)
        teacher_dist = F.softmax(teacher_logits / T, dim=1)

        return nn.KLDivLoss(reduction="batchmean")(student_dist, teacher_dist) * (T**2)

    def _compute_class_means(self):
        self.model.eval()

        for c in self.exemplars:
            class_features = []
            # extract and normalize features
            for feature, _ in self.exemplars[c]:
                with torch.no_grad():
                    feat = self.model.feature_extractor(feature.unsqueeze(0))
                    feat = torch.nn.functional.normalize(feat, p=2, dim=1)
                    class_features.append(feat)

            # compute mean of class features
            class_features = torch.cat(class_features, dim=0)
            class_mean = torch.mean(class_features, dim=0)
            class_mean = torch.nn.functional.normalize(class_mean, p=2, dim=0)

            self.exemplar_means[c] = class_mean